In [291]:
import pandas as pd
import requests
import json
from pprint import pprint
import numpy as np
import hvplot.pandas
import gmaps
import os
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import datetime as dt
import pytz
%matplotlib inline

In [213]:
# Import Flood Data
files = '../../Resources/Flood/UPDATED_CLEAN_NFP_CLAIMS.csv'
flood_df = pd.read_csv(files)
flood_df = flood_df.dropna()

In [215]:
#Convert Columns to DateTime Type
flood_df['dateOfLoss'] = pd.to_datetime(flood_df['dateOfLoss'].str.slice(0,10,1),format='%Y/%m/%d')
flood_df['asOfDate'] = pd.to_datetime(flood_df['asOfDate'].str.slice(0,10,1),format='%Y/%m/%d')
flood_df['originalConstructionDate'] = pd.to_datetime(flood_df['originalConstructionDate'].str.slice(0,10,1),format='%Y/%m/%d')
flood_df['originalNBDate'] = pd.to_datetime(flood_df['originalNBDate'].str.slice(0,10,1),format='%Y/%m/%d')

#Don't Need this Column so we going to delete it!
del flood_df['asOfDate']


#Filter Data on Properties with Primary Residence
flood_df=flood_df.loc[flood_df['primaryResidence'] == 1].reset_index(drop=True)

In [216]:
#Rename Columns
flood_df = flood_df.rename(columns={'dateOfLoss':'Date Of Loss',
                                    'originalConstructionDate':'Building Construction Date',
                                    'originalNBDate':'Flood Policy Date',
                                   'numberOfFloorsInTheInsuredBuilding':'Numbers of Floors',
                                   'policyCount':'Number of Policies','reportedCity':'City'})

In [217]:
#Create new Column Called "Date of Loss" & "Flood Policy Date" and add it to Dataframe!
YearOfLoss = []
FloodPolicyYear = []

for yearsLoss in flood_df['Date Of Loss']:
    YearOfLoss.append(yearsLoss.year)
    
for yearsPolicy in flood_df['Flood Policy Date']:
    FloodPolicyYear.append(yearsPolicy.year)    
    
flood_df['Year Of Loss'] = YearOfLoss
flood_df['Flood Policy Year'] = FloodPolicyYear


#Only Want Paid Claims
#paid_Claims = flood_df[flood_df['amountPaidOnBuildingClaim'] > 0].reset_index(drop=True)

#Convert Scientific Notation
pd.options.display.float_format = '{:,.2f}'.format



#Export Newly Clean Data
flood_df.to_csv('updated_clean_data.csv',index=False,header=True)

In [218]:
# Flood Data Severity  (2010-2019) - BASED OFF OF CLAIMS
flood_severity = flood_df.groupby(['Year Of Loss']).sum().sort_values(by='Year Of Loss',ascending=False)['amountPaidOnBuildingClaim']

In [219]:
# Flood Data Severity (2010-2019) - BASED OFF OF COVERAGE AMOUNT(PROPERTY OF VALUES)
flood_severity_Coverage = flood_df.groupby(['Year Of Loss']).sum().sort_values(by='Year Of Loss',ascending=False)['totalBuildingInsuranceCoverage']

In [220]:
#Frequency of Flood Claims per Year
paid_Claims_Year = paid_Claims.groupby(['Year Of Loss']).count().sort_values(by='Year Of Loss',ascending=False)['amountPaidOnBuildingClaim']

In [221]:
#Average Cost Per Claim (flood_severity / paid_Claims_Year)
Average_Cost_per_claim = flood_severity / paid_Claims_Year

In [222]:
#Average cost of Claims Per Year (flood_severity_Coverage - flood_severity)
Average_Cost_per_claim_v2 =(flood_severity_Coverage - flood_severity)/ paid_Claims_Year

In [223]:
#LOSS RATIO (80%) = (Average_Cost_per_claim * 0.8) + Average_Cost_per_claim
Premium_per_claim = (Average_Cost_per_claim * 0.2) + Average_Cost_per_claim

In [224]:
#LOSS RATIO (80%) = (Average_Cost_per_claim * 0.8) + Average_Cost_per_claim (BASED ON COVERAGE - PAID COVERAGE)
Premium_per_claim_v2 = (Average_Cost_per_claim_v2 * 0.2) + Average_Cost_per_claim_v2

In [225]:
flood_severity_Coverage

Year Of Loss
2018     3540055200
2017    13157665900
2016     7412012900
2015     2942191400
2014     2370542000
2013     2389655900
2012    22576007300
2011     9376820900
2010     3957438100
2009     4146305200
2008     6219449000
2007     2372977400
2006     1965144200
2005    17937718500
2004     5519687600
2003     1236525600
Name: totalBuildingInsuranceCoverage, dtype: int64

In [226]:
flood_severity

Year Of Loss
2018     554,837,921.09
2017   3,673,945,852.28
2016   1,500,736,416.31
2015     284,294,984.28
2014     150,868,969.22
2013     211,189,804.19
2012   3,949,542,309.38
2011     857,699,218.14
2010     277,066,873.07
2009     346,906,662.86
2008   1,133,773,583.24
2007     182,885,391.77
2006     284,926,788.18
2005   6,287,686,582.89
2004     742,480,922.38
2003      91,813,575.95
Name: amountPaidOnBuildingClaim, dtype: float64

In [227]:
flood_severity_Coverage - flood_severity

Year Of Loss
2018    2,985,217,278.91
2017    9,483,720,047.72
2016    5,911,276,483.69
2015    2,657,896,415.72
2014    2,219,673,030.78
2013    2,178,466,095.81
2012   18,626,464,990.62
2011    8,519,121,681.86
2010    3,680,371,226.93
2009    3,799,398,537.14
2008    5,085,675,416.76
2007    2,190,092,008.23
2006    1,680,217,411.82
2005   11,650,031,917.11
2004    4,777,206,677.62
2003    1,144,712,024.05
dtype: float64

In [228]:
paid_Claims_Year

Year Of Loss
2018    14190
2017    49736
2016    27848
2015     8910
2014     4928
2013     6619
2012    73191
2011    33461
2010    12000
2009    12396
2008    28606
2007     9197
2006     9252
2005    89532
2004    19636
2003    12740
Name: amountPaidOnBuildingClaim, dtype: int64

In [229]:
#'Average Cost Per Claim PER YEAR: LEFT OVER COVERAGE'
Average_Cost_per_claim_v2.hvplot(title='Average Investment Per Year' )

:Curve   [Year Of Loss]   (0)

In [230]:
Average_Cost_per_claim.hvplot(title='Average Cost Per Claim PER YEAR: BASED OFF OF PAID CLAIMS')

:Curve   [Year Of Loss]   (amountPaidOnBuildingClaim)

In [231]:
#Export Flood Severity to CSV
flood_severity.to_csv('Flood_severity.csv',index=True,header=True)

In [232]:
flood_severity.hvplot(title='Serverity: Total $ Amount of Claims')

:Curve   [Year Of Loss]   (amountPaidOnBuildingClaim)

In [233]:
#Frequency
paid_Claims_Year

Year Of Loss
2018    14190
2017    49736
2016    27848
2015     8910
2014     4928
2013     6619
2012    73191
2011    33461
2010    12000
2009    12396
2008    28606
2007     9197
2006     9252
2005    89532
2004    19636
2003    12740
Name: amountPaidOnBuildingClaim, dtype: int64

In [234]:
Premium_per_claim_v2.hvplot(title='Premium Average Cost')

:Curve   [Year Of Loss]   (0)

In [235]:
Average_Cost_per_claim

Year Of Loss
2018   39,100.63
2017   73,868.95
2016   53,890.28
2015   31,907.41
2014   30,614.64
2013   31,906.60
2012   53,962.13
2011   25,632.80
2010   23,088.91
2009   27,985.37
2008   39,634.12
2007   19,885.33
2006   30,796.24
2005   70,228.37
2004   37,812.23
2003    7,206.72
Name: amountPaidOnBuildingClaim, dtype: float64

In [236]:
Premium_per_claim_v2 - Average_Cost_per_claim_v2

Year Of Loss
2018   42,074.94
2017   38,136.24
2016   42,453.87
2015   59,660.97
2014   90,084.13
2013   65,824.63
2012   50,898.24
2011   50,919.71
2010   61,339.52
2009   61,300.40
2008   35,556.70
2007   47,626.23
2006   36,321.17
2005   26,024.29
2004   48,657.64
2003   17,970.36
dtype: float64

In [237]:
Premium_per_claim - Average_Cost_per_claim

Year Of Loss
2018    7,820.13
2017   14,773.79
2016   10,778.06
2015    6,381.48
2014    6,122.93
2013    6,381.32
2012   10,792.43
2011    5,126.56
2010    4,617.78
2009    5,597.07
2008    7,926.82
2007    3,977.07
2006    6,159.25
2005   14,045.67
2004    7,562.45
2003    1,441.34
Name: amountPaidOnBuildingClaim, dtype: float64

In [238]:
paid_Claims.tail(75)

,City,Number of Policies,communityRatingSystemDiscount,Date Of Loss,floodZone,latitude,longitude,Numbers of Floors,occupancyType,Building Construction Date,Flood Policy Date,amountPaidOnBuildingClaim,state,totalBuildingInsuranceCoverage,primaryResidence,Year Of Loss,Flood Policy Year
412167,SCITUATE,1.00,0.00,2003-01-04,V02,42.20,-70.70,2.00,1.00,1920-05-21,2002-04-19,"7,760.57",MA,"75,400.00",1.00,2003,2002
412168,WINTHROP,1.00,0.00,2003-01-04,B,42.40,-71.00,2.00,1.00,1900-01-01,1997-05-26,"36,232.59",MA,"140,000.00",1.00,2003,1997
412169,SCITUATE,1.00,0.00,2003-01-04,C,42.20,-70.70,3.00,1.00,1985-01-01,1998-10-27,"2,880.81",MA,"183,800.00",1.00,2003,1998
412170,WINTHROP,1.00,0.00,2003-01-04,B,42.40,-71.00,3.00,1.00,1946-07-01,1998-09-04,"4,493.77",MA,"170,000.00",1.00,2003,1998
412171,HUMAROCK,1.00,0.00,2003-01-04,V02,42.10,-70.70,2.00,1.00,1979-02-01,1999-05-23,"1,473.21",MA,"200,000.00",1.00,2003,1999
412172,HUMAROCK,1.00,0.00,2003-01-04,A04,42.10,-70.70,2.00,1.00,1949-01-01,1999-10-29,"4,384.29",MA,"121,000.00",1.00,2003,1999
412173,MARSHFIELD,1.00,0.00,2003-01-04,A02,42.10,-70.60,2.00,1.00,1996-01-01,2000-07-28,490.51,MA,"140,000.00",1.00,2003,2000
412174,HUMAROCK,1.00,0.00,2003-01-04,V03,42.10,-70.70,3.00,1.00,1978-01-01,2001-02-18,"5,827.17",MA,"185,000.00",1.00,2003,2001
412175,SAUNDERSTOWN,1.00,0.00,2003-01-04,C,41.50,-71.50,3.00,1.00,2002-05-25,2002-08-18,"2,101.54",RI,"250,000.00",1.00,2003,2002
412176,WINTHROP,1.00,0.00,2003-01-04,C,42.40,-71.00,2.00,1.00,1945-01-01,2002-06-28,"4,446.83",MA,"47,000.00",1.00,2003,2002


In [239]:
# Flood Data per State (2010-2019)
flood_df.groupby(['state']).sum().sort_values(by='amountPaidOnBuildingClaim',ascending=False)

,Number of Policies,communityRatingSystemDiscount,latitude,longitude,Numbers of Floors,occupancyType,amountPaidOnBuildingClaim,totalBuildingInsuranceCoverage,primaryResidence,Year Of Loss,Flood Policy Year
state,,,,,,,,,,,
LA,122924,0,"3,618,729.80","-10,623,366.50",154554,147870,"6,473,461,121.54",15863120100,118341,237544713,236903693
TX,88580,0,"2,567,838.80","-7,723,636.70",118299,93019,"4,337,377,921.12",15582482700,82801,166708995,166319045
NY,63116,0,"2,143,524.90","-4,121,411.20",123264,67853,"2,195,294,081.20",13016629400,54102,108804549,108460056
NJ,71159,0,"2,050,957.10","-3,910,274.20",112318,70270,"2,039,061,726.12",14432599000,51729,104032571,103694864
FL,128377,0,"2,021,729.90","-5,567,424.00",106252,83372,"1,616,231,398.69",21604297700,66926,134518362,134200728
NC,22161,0,"698,556.60","-1,566,090.60",38236,23553,"520,798,214.30",3790415300,19797,39852990,39725663
MS,11028,0,"312,035.20","-892,493.00",13547,12202,"456,334,137.58",1408959000,9980,20032603,19989225
PA,13093,0,"468,058.90","-1,015,235.50",29254,18967,"338,297,556.42",1693359000,12475,25057567,24996241
CT,8770,0,"297,237.40","-573,430.90",17304,9468,"263,147,925.90",1728939900,7471,15022338,14967334


### HEATMAP PLOT!!!!

In [240]:
from config import (gkey)

In [243]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [287]:
# Store 'Lat' and 'Lng' into  locations 
locations = flood_df[["latitude", "longitude"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
paid_on_claim = flood_df["totalBuildingInsuranceCoverage"].astype(float).abs()

In [288]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=paid_on_claim, 
                                 dissipating=False, max_intensity=100000,
                                 point_radius = 0.1)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 100000
heat_layer.point_radius = 0.1


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))